In [1]:
# ✅ Step 1: Install dependencies
!pip install transformers datasets -q


In [2]:
# ✅ Step 2: Disable W&B logging (optional for speed)
import os
os.environ["WANDB_DISABLED"] = "true"


In [3]:
# ✅ Step 3: Import necessary libraries
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score
import torch


In [4]:
# ✅ Step 4: Load the AG News dataset
dataset = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:
# ✅ Step 5: Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [6]:
# ✅ Step 6: Tokenize the text
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [7]:
# ✅ Step 7: Select a smaller dataset for faster training
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
val_dataset = tokenized_datasets["test"].select(range(200))

In [8]:
# ✅ Step 8: Load BERT model for classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# ✅ Step 9: Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

In [11]:
# ✅ Step 10: Set training parameters
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch", # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Faster training
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="no"  # Don't save intermediate checkpoints
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
# ✅ Step 11: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-14-4158931840.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
# ✅ Step 13: Evaluate the model
results = trainer.evaluate()

In [16]:
# ✅ Step 14: Display Results
print("\n📊 Final Evaluation Metrics:")
print(f"✅ Accuracy     : {results['eval_accuracy']:.4f}")
print(f"✅ F1 Score     : {results['eval_f1']:.4f}")
print(f"🕒 Eval Runtime : {results['eval_runtime']:.2f} seconds")
print(f"📈 Samples/Sec  : {results['eval_samples_per_second']:.2f}")



📊 Final Evaluation Metrics:
✅ Accuracy     : 0.3050
✅ F1 Score     : 0.1426
🕒 Eval Runtime : 67.02 seconds
📈 Samples/Sec  : 2.98
